In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [4]:
#Read the dataset file
data = pd.read_csv('/kaggle/input/iriscsv/Iris.csv', index_col = 0)

In [5]:
#Display the first five rows of the dataframe
print(data.head())

    SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
Id                                                                       
1             5.1           3.5            1.4           0.2  Iris-setosa
2             4.9           3.0            1.4           0.2  Iris-setosa
3             4.7           3.2            1.3           0.2  Iris-setosa
4             4.6           3.1            1.5           0.2  Iris-setosa
5             5.0           3.6            1.4           0.2  Iris-setosa


In [6]:
#Displaying the information of the dataframe
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 150 entries, 1 to 150
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SepalLengthCm  150 non-null    float64
 1   SepalWidthCm   150 non-null    float64
 2   PetalLengthCm  150 non-null    float64
 3   PetalWidthCm   150 non-null    float64
 4   Species        150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 7.0+ KB
None


In [7]:
print(data.shape)

(150, 5)


In [8]:
# check there is no null values and check for missing values
print(data.isna().sum())

SepalLengthCm    0
SepalWidthCm     0
PetalLengthCm    0
PetalWidthCm     0
Species          0
dtype: int64


In [10]:
#Check for duplicates
print(data.duplicated(keep = False).value_counts())
print(data[data.duplicated(keep = False)])

False    145
True       5
Name: count, dtype: int64
     SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm         Species
Id                                                                           
10             4.9           3.1            1.5           0.1     Iris-setosa
35             4.9           3.1            1.5           0.1     Iris-setosa
38             4.9           3.1            1.5           0.1     Iris-setosa
102            5.8           2.7            5.1           1.9  Iris-virginica
143            5.8           2.7            5.1           1.9  Iris-virginica


In [11]:
#Drop duplicates
data = data.drop_duplicates(keep = 'first').copy()
print(data.shape)

(147, 5)


In [13]:
#Reset the index of the dataframe

data = data.reset_index(drop = True)

In [15]:
#Split the dataframe into X(features) and y(target or labels)
X = data.iloc[:, 0:4].values
y = data['Species'].values

In [16]:
print(X.shape)
print(y.shape)

(147, 4)
(147,)


In [18]:
#Normalization on X
scaler = MinMaxScaler()
scaler.fit(X)
X_scaled = scaler.fit_transform(X)


In [20]:
#Convert target using LabelEncoder
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

In [22]:
#Categorically encoding the labels

num_categories = 3
y_onehot = keras.utils.to_categorical(y_encoded, num_categories)

In [23]:
#convert X_scaled and y_onehot into train and valid data

X_train, X_valid, y_train, y_valid = train_test_split(X_scaled, y_onehot,
                                                     test_size = 0.2,
                                                     random_state = 42)

In [24]:
#Define a model

model = Sequential()
model.add(Dense(units = 512, activation = 'relu', input_shape = (4,)))
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dense(units = 512, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = num_categories, activation = 'softmax'))

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               2560      
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 512)               262656    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 3)                 1539      
                                                                 
Total params: 529411 (2.02 MB)
Trainable params: 529411 (2.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
#Compiling the model

model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [27]:
history=model.fit(X_train, y_train,
          epochs = 100,
          verbose = 1,
          validation_data = (X_valid, y_valid))

Epoch 1/100
4/4 [==============================] - 1s 116ms/step - loss: 0.9893 - accuracy: 0.4530 - val_loss: 0.8050 - val_accuracy: 0.6667
Epoch 2/100
4/4 [==============================] - 0s 24ms/step - loss: 0.6599 - accuracy: 0.7179 - val_loss: 0.5551 - val_accuracy: 0.8333
Epoch 3/100
4/4 [==============================] - 0s 21ms/step - loss: 0.4713 - accuracy: 0.7949 - val_loss: 0.4725 - val_accuracy: 0.6667
Epoch 4/100
4/4 [==============================] - 0s 20ms/step - loss: 0.5725 - accuracy: 0.6581 - val_loss: 0.5058 - val_accuracy: 0.6667
Epoch 5/100
4/4 [==============================] - 0s 23ms/step - loss: 0.4385 - accuracy: 0.7607 - val_loss: 0.5078 - val_accuracy: 0.7000
Epoch 6/100
4/4 [==============================] - 0s 24ms/step - loss: 0.5035 - accuracy: 0.6923 - val_loss: 0.4066 - val_accuracy: 0.7000
Epoch 7/100
4/4 [==============================] - 0s 23ms/step - loss: 0.3939 - accuracy: 0.8034 - val_loss: 0.3511 - val_accuracy: 0.9000
Epoch 8/100
4/4 [==

In [28]:
#Print actual and predicted value

y_pred = model.predict(X_valid)

actual = np.argmax(y_valid, axis = 1)
predicted = np.argmax(y_pred, axis = 1)

print(f"Actual:    {actual}")
print(f"Predicted: {predicted}")



1/1 [==============================] - 0s 121ms/step
Actual:    [2 1 2 0 2 0 1 0 1 0 0 1 2 1 2 1 0 1 2 0 0 2 0 2 1 0 1 2 1 0]
Predicted: [2 1 2 0 2 0 1 0 2 0 0 1 2 1 2 2 0 1 2 0 0 2 0 2 1 0 1 2 1 0]
